In [4]:
'''
Predicting the output of the test image (same as training image). The steps here are largely analogous to the ones
in 'train.py' and could in future iterations of this work be put into a reusable function (one could also just use
'rgb_batch' in this special case of train and test data being the same)
'''

import torch 
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image
import numpy as np

from train import *

# Test image
test_image = Image.open("rgb.png")

# Now, the same per-processing steps have to be taken

test_array = np.array(test_image).reshape(test_image.size[0],test_image.size[1],4)

test_array_trans = test_array[np.newaxis,:,:,:3]
test_array_trans = np.einsum('ijkl->ilkj',test_array_trans)

cut_value_width = int(rgb_array_trans.shape[3] % 64 /2)
cut_value_height = int(rgb_array_trans.shape[2] % 64 /2)

test_array_trans = test_array_trans[:,:,cut_value_height:test_array_trans.shape[2] -cut_value_height, 
                                  cut_value_width:test_array_trans.shape[3] - cut_value_width]

number_patches_height = (test_array_trans.shape[2]/64)
number_patches_width = (test_array_trans.shape[3]/64)

test_batch = np.zeros((int(number_patches_height*number_patches_width),3,64,64))

patch_counter = 0

for i in range(int(number_patches_height)):
    for j in range(int(number_patches_width)):
        test_batch[patch_counter,:,:64,:64] = rgb_array_trans[:,:,i*64:(i+1)*64,j*64:(j+1)*64]

        patch_counter += 1
        
        
# Predicting the classes using the network. First, we create a prediction_batch to store the patch-predictions

y = torch.zeros((test_batch.shape[0],2,64,64))

for d in range(test_batch.shape[0]):
    x = torch.tensor(rgb_batch[d:d+1,:,:,:], dtype = torch.double)
    
    y[d,:,:,:] = train.model(x)
    
    

'''
Possible further steps: 
- cross validation to determine the value of certain hyper-parameters like the learning rate or the number of epochs
- training on a GPU instead of a CPU
- training with a larger data-set
- trying some of the augmentations to the Fully Convolutional Network as mentioned in papers based on the one covering
    FCCs
- higher degree of automization in the image-preprocessing (especially for a possible larger data-set)
- visualization of the dense segmentation (mirroring the input image)
- 

'''

ModuleNotFoundError: No module named 'train'